# 0. Libraries:

In [40]:
import os
from typing import List, Dict
import fitz
from tqdm.auto import tqdm
import pandas as pd

import numpy as np
import torch

device = torch.device("cpu")

In [ ]:
nltk.download("punkt", quiet=True)

# 1. PDF file processing

In [ ]:
pdf_path = 'The Ultimate Guide to Tarot - A Beginner.pdf'

def text_formatting(text: str) -> str:
    prepared_text = text.replace("\n", " ").strip()

    return prepared_text

def open_and_read(pdf_path: str) -> List[Dict]:
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatting(text)
        pages_and_texts.append({"page_number": page_number - 6,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,
                                "text": text})
    return pages_and_texts
pages_and_text = open_and_read(pdf_path=pdf_path)
pages_and_text[:3]

0it [00:00, ?it/s]

[{'page_number': -6,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''},
 {'page_number': -5,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''},
 {'page_number': -4,
  'page_char_count': 137,
  'page_word_count': 26,
  'page_sentence_count_raw': 1,
  'page_token_count': 34.25,
  'text': 'A Beginner’s Guide to the Cards, Spreads, and Revealing the Mystery of the Tarot LIZ DEAN author of The Art of Tarot and The Golden Tarot'}]

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-7,0,1,1,0.00,
1,-6,0,1,1,0.00,
2,-5,137,26,1,34.25,"A Beginner’s Guide to the Cards, Spreads, and ..."
3,-4,443,79,1,110.75,Contents CHAPTER 1 INTRODUCING THE TAROT CHAPT...
4,-3,491,95,1,122.75,"XIX, The Sun XX, Judgment XXI, The World CHAPT..."


In [ ]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,493.00,493.00,493.00,493.00,493.00
mean,239.00,981.51,169.57,8.21,245.38
std,142.46,912.48,158.65,8.10,228.12
min,-7.00,0.00,1.00,1.00,0.00
25%,116.00,145.00,26.00,1.00,36.25
50%,239.00,605.00,106.00,4.00,151.25
75%,362.00,1839.00,314.00,16.00,459.75
max,485.00,2632.00,461.00,25.00,658.00


## 1.2 Create data chunks

In [ ]:
import re

def simple_sent_tokenize(text: str) -> List[str]:
    # Step 1: Split into sentences
    sentences = re.split(r'(?<=[.!?;])\s+', text)

    # Step 2: Merge headers (all caps, no punctuation, short length)
    merged = []
    skip_next = False
    for i, sent in enumerate(sentences):
        if skip_next:
            skip_next = False
            continue

        if (sent.isupper() and len(sent.split()) <= 6 and not re.search(r'[.!?;]', sent)):
            # Merge with next sentence if available
            if i + 1 < len(sentences):
                merged.append(sent + " " + sentences[i+1])
                skip_next = True
            else:
                merged.append(sent)
        else:
            merged.append(sent)

    return merged

def prepare_chunks(pages_and_texts: List[Dict],
                   min_chars: int = 250,
                   max_chars: int = 1500,
                   overlap_sentences: int = 2) -> List[Dict]:
    chunks = []
    buffer = None  # holds short pages to merge

    for page in pages_and_texts:
        text = page["text"].strip()
        if not text:
            continue  # skip empty

        # Merge short pages
        if len(text) < min_chars:
            if buffer:
                buffer["text"] += " " + text
            else:

                buffer = {"page_number": page["page_number"], "text": text, "page_token_count": (len(text) / 4)}
            continue

        # Flush buffered short page
        if buffer:
            chunks.append(buffer)
            buffer = None

        # Split into sentences using regex
        sentences = simple_sent_tokenize(text)

        current_chunk = []
        current_length = 0

        for i, sentence in enumerate(sentences):
            if current_length + len(sentence) > max_chars and current_chunk:
                # save current chunk
                chunk_text = " ".join(current_chunk)
                chunks.append({"page_number": page["page_number"], "text": chunk_text, "page_token_count": (len(chunk_text) / 4)})

                # start new chunk with overlap sentences
                overlap = current_chunk[-overlap_sentences:] if overlap_sentences else []
                current_chunk = overlap + [sentence]
                current_length = sum(len(s) for s in current_chunk)
            else:
                current_chunk.append(sentence)
                current_length += len(sentence)

        # Save last chunk
        if current_chunk:

            chunks.append({"page_number": page["page_number"], "text": " ".join(current_chunk), "page_token_count": (len(" ".join(current_chunk)) / 4)})

    # Flush leftover buffer
    if buffer:
        chunks.append(buffer)

    return chunks


In [ ]:
chunks = prepare_chunks(pages_and_texts[7:])

print(len(chunks))          # number of embedding-ready chunks
print(chunks[0]["text"])

570
1  INTRODUCING THE TAROT HOW TO WORK WITH THIS BOOK This book offers everything you need to know to read tarot cards for daily affirmation, prediction, and intuitive and spiritual development. Anyone can learn to work with tarot and can benefit greatly from its insights; all you need is an open mind and a willingness to trust the impressions you sense during a reading. We begin with the basics—the structure of a tarot deck, and how the cards link with astrology, Kabbala, and numbers (see pages 10 and 19). Next, you’ll learn how to lay out the cards for a reading. You can try the traditional spreads on pages 20–28, and the mini-layouts given for each major arcana card—a total of thirty. The mini-layouts are all original to this book, and I invite you to share them and use them as inspiration for devising your own spreads; the aim is to help you be creative with tarot, to experiment and find ways to read the cards that work for you. There’s no right or wrong way to lay out cards just

In [ ]:
pd.DataFrame(chunks).describe().round(2)

,page_number,page_token_count
count,570.00,570.00
mean,239.66,230.06
std,139.14,124.99
min,0.00,15.50
25%,120.50,105.25
50%,240.50,257.00
75%,359.50,356.94
max,485.00,378.75


In [ ]:
for i in range(len(chunks)):
    chunks[i]["chunk_word_count"] = len([word for word in chunks[i]["text"].split(" ")])


In [ ]:
df = pd.DataFrame(chunks)#.describe().round(2)

In [ ]:
df

,page_number,text,page_token_count,chunk_word_count
0,0,1 INTRODUCING THE TAROT HOW TO WORK WITH THIS...,376.50,270
1,0,In the detailed major arcana card interpretati...,202.00,135
2,1,The minor arcana interpretations are divided b...,344.25,226
3,1,HOW READING TAROT CAN BENEFIT YOU Tarot is a s...,248.50,156
4,2,The tarot deck shown throughout this book is t...,330.75,214
...,...,...,...,...
565,481,"Hermit, The, 69 Hierophant, The, 53 High Pries...",175.00,119
566,482,"Ten of Swords, 194–195 Page of Swords, 196–197...",170.25,112
567,483,"Three of Swords, 180–181 Three of Wands, 208–2...",134.25,92
568,484,This book is dedicated to all students of taro...,28.75,20


In [ ]:
df.to_csv("tarot_text.csv", index = False)

# 2. Embedding text chunks

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")

2025-09-17 13:39:26.250837: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WA

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
%%time

for item in tqdm(chunks):
    item["embedding"] = embedding_model.encode(item["text"])

  0%|          | 0/570 [00:00<?, ?it/s]

CPU times: user 4min 55s, sys: 19 s, total: 5min 14s
Wall time: 4min 55s


In [ ]:
text_chunks_and_embeddings_df = pd.DataFrame(chunks)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
text_chunks_and_embeddings_df

,page_number,text,page_token_count,chunk_word_count,embedding
0,0,1 INTRODUCING THE TAROT HOW TO WORK WITH THIS...,376.50,270,"[0.03744885, -0.058901116, -0.03133846, 0.0446..."
1,0,In the detailed major arcana card interpretati...,202.00,135,"[0.029477408, -0.040557835, -0.046912964, 0.05..."
2,1,The minor arcana interpretations are divided b...,344.25,226,"[0.029125197, -0.042218585, -0.02380663, 0.046..."
3,1,HOW READING TAROT CAN BENEFIT YOU Tarot is a s...,248.50,156,"[-0.017528765, -0.0047595724, -0.023500122, 0...."
4,2,The tarot deck shown throughout this book is t...,330.75,214,"[0.058840264, -0.043807305, -0.011972893, 0.04..."
...,...,...,...,...,...
565,481,"Hermit, The, 69 Hierophant, The, 53 High Pries...",175.00,119,"[0.05572427, -0.023874342, 0.00534033, 0.06752..."
566,482,"Ten of Swords, 194–195 Page of Swords, 196–197...",170.25,112,"[0.05402895, -0.03954698, 0.0053145206, 0.0208..."
567,483,"Three of Swords, 180–181 Three of Wands, 208–2...",134.25,92,"[0.078784734, -0.061991517, -0.0022283802, 0.0..."
568,484,This book is dedicated to all students of taro...,28.75,20,"[0.0469173, -0.0035335498, -0.016104946, 0.040..."


# 3. RAG

In [2]:
# if start new
import random
import torch
import numpy as np
import pandas as pd

device = torch.device("cuda")

text_chunks_and_embeddings_df = pd.read_csv("text_chunks_and_embeddings_df.csv")


text_chunks_and_embeddings_df["embedding"] = text_chunks_and_embeddings_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))


chunks = text_chunks_and_embeddings_df.to_dict(orient="records")


embeddings = torch.tensor(np.array(text_chunks_and_embeddings_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([570, 768])

In [5]:
embeddings = torch.tensor(np.stack(text_chunks_and_embeddings_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([570, 768])

In [6]:
embeddings

tensor([[ 0.0374, -0.0589, -0.0313,  ...,  0.0308,  0.0246,  0.0085],
        [ 0.0295, -0.0406, -0.0469,  ...,  0.0059, -0.0042,  0.0014],
        [ 0.0291, -0.0422, -0.0238,  ...,  0.0460,  0.0122,  0.0069],
        ...,
        [ 0.0788, -0.0620, -0.0022,  ..., -0.0173, -0.0124,  0.0089],
        [ 0.0469, -0.0035, -0.0161,  ...,  0.0501,  0.0528,  0.0030],
        [ 0.0539,  0.0514, -0.0294,  ...,  0.0232,  0.0183, -0.0421]],
       device='cuda:0')

## 3.1 Embedding model

In [3]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
# 1. Define the query
# Note: This could be anything. But since we're working with a nutrition textbook, we'll stick with nutrition-based queries.
query = "the moon card reversed!"
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples
# Note: It's important to embed your query with the same model you embedded your examples with.
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

# 3. Get similarity scores with the dot product
from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# 4. Get the top-k results (we'll keep this to 5)
top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Query: the moon card reversed!
Time take to get scores on 570 embeddings: 0.00024 seconds.


torch.return_types.topk(
values=tensor([0.6461, 0.6137, 0.5700, 0.5389, 0.5273], device='cuda:0'),
indices=tensor([232, 228, 235, 229, 240], device='cuda:0'))

In [15]:
text_chunks_and_embeddings_df.text[232]

'When the Moon is reversed, you may avoid difficult emotions and confrontations, so your needs are not expressed or recognized. A trauma is ignored again rather than explored, so the Moon reversed can show you going back to old ways of coping with the past. The card can also show you feeling stuck in an old emotional pattern that keeps arising—until you give it attention. ITS WISDOM MESSAGE Be guided by messages from your unconscious. THE MOON’S SYMBOLS In the Rider-Waite tarot, the Moon appears with these magical symbols. Some of them reappear in other major arcana cards, so learn to recognize them and you’ll soon find you can apply your knowledge throughout the deck. The crayfish: This creature is a symbol of the primal self, which in this environment is not at peace and is struggling to surface, like subconscious fears. The wolf and dog: The canines show fear of the unknown. They also are guardians of experience, representing a rite of passage. The wolf is wild instinct and the dog,

In [ ]:
embeddings

tensor([[ 0.0374, -0.0589, -0.0313,  ...,  0.0308,  0.0246,  0.0085],
        [ 0.0295, -0.0406, -0.0469,  ...,  0.0059, -0.0042,  0.0014],
        [ 0.0291, -0.0422, -0.0238,  ...,  0.0460,  0.0122,  0.0069],
        ...,
        [ 0.0788, -0.0620, -0.0022,  ..., -0.0173, -0.0124,  0.0089],
        [ 0.0469, -0.0035, -0.0161,  ...,  0.0501,  0.0528,  0.0030],
        [ 0.0539,  0.0514, -0.0294,  ...,  0.0232,  0.0183, -0.0421]])

In [4]:

def retrieve(query):

    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
    top_results_dot_product = torch.topk(dot_scores, k=5)
    top_indices = top_results_dot_product.indices.cpu().tolist()
    response = text_chunks_and_embeddings_df.text.iloc[top_indices].tolist()

    return response



In [5]:
retrieve("Sun reversed")[0][:500]

'Here’s some other good news the Sun can predict:  Home: You feel comfortable and secure in your home—you may also feel more like entertaining others. In particular, the Sun shows children coming into your home. Relationships: Partnerships bloom under the sun as your relationship grows and you enjoy every minute together. You may also take a trip away to a sunny place to escape everyday pressures. Career and money: The Sun does not specifically predict money but does show success and a position f'

# 4. LLM for local generation

In [ ]:
pip install sentence-transformers faiss-cpu transformers torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached faiss_cpu-1.8.0.post1-cp38-cp38-macosx_10_14_x86_64.whl.metadata (3.7 kB)
Using cached faiss_cpu-1.8.0.post1-cp38-cp38-macosx_10_14_x86_64.whl (7.3 MB)
    opencv-python (>=3.) ; extra == 'all'
                  ~~~~^
Note: you may need to restart the kernel to use updated packages.


In [6]:
from huggingface_hub import login
login(


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # automatically uses GPU
    torch_dtype=torch.float16
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [12]:
user_question = "i recevied the sun card, when asking about my love future"
retrieved_context = "\n".join(retrieve(user_question))

prompt = f"""
You are a mystical tarot reader.
Using only the context provided, answer the question in insightful way.

Context:
{retrieved_context}

Question:
{user_question}

Answer:
"""

In [14]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=200,      # adjust length of answer
    do_sample=True,          # sampling for more creative output
    temperature=0.7,         # randomness
    top_p=0.9,               # nucleus sampling
    repetition_penalty=1.1   # avoid repetition
)

# Decode the generated tokens
generated_tokens = outputs[0][inputs["input_ids"].shape[1]:]

# Decode the generated part
answer = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("\n--- Tarot Reading ---\n")
print(answer)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



--- Tarot Reading ---

The Sun card represents the most positive aspects of your life. In this case, it is a sign that you will be able to find the love of your life. You will meet someone who will complement and fulfill you. You are in a good position to meet the person of your dreams and build a strong relationship with them. This is an auspicious time for new beginnings and relationships. This card is associated with joy, happiness, and a feeling of contentment. It is a time to let yourself be open to new experiences and possibilities. You may meet a new lover, or a new friend, who will be able to enrich your life. The Sun card also indicates that you will have a lot of creative energy, which can be a great way to meet new people. This is a good time to explore your creative talents and find ways to apply them in your daily life. The Sun card also suggests that you will be able to attract new opportunities and have a lot of luck. It is a good


In [34]:
def tarot_reader(user_question):
    retrieved_context = "\n".join(retrieve(user_question))

      prompt = f"""
      You are a mystical tarot reader.
      Using only the context provided, answer the question in insightful and clear way.
      When asked about more then one card, provide the information, regarding card combinations.
      Never give unfinished sentences. Provide information only about mentioned cards.

      Context:
      {retrieved_context}

      Question:
      {user_question}

      Answer:
      """
      inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

      outputs = model.generate(
      **inputs,
      max_new_tokens=100,      # adjust length of answer
      do_sample=True,          # sampling for more creative output
      temperature=0.7,         # randomness
      top_p=0.9,               # nucleus sampling
      repetition_penalty=1.1   # avoid repetition
    )

    # Decode the generated tokens
    generated_tokens = outputs[0][inputs["input_ids"].shape[1]:]

    # Decode the generated part
    answer = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return answer


In [35]:
user_question = input("Ask your tarot question: ")
print(tarot_reader(user_question))

Ask your tarot question: i asked about love and received  two of wands reversed


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 The card represents a romantic partnership. The reversed Page of Wands is often a sign of a short-lived affair. You may meet someone with whom you have a strong connection, but the relationship will not be long-term. This card can also show a long-term relationship, but it may not be one that is happy. The card can also show a relationship that has been going on for a long time. It is a sign of an old-fashioned marriage, where both partners are committed


In [42]:
user_question = input("Ask your tarot question: ")
print(tarot_reader(user_question))

Ask your tarot question: what zodiac sign is associated with the sun card


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 Leo
The Sun is associated with Leo, a fire sign, and its glyph is a single, large letter L.
A figure in a long robe, standing in front of a group of trees, is the image of the Sun. The figure is standing on a single, large, white stone. The trees behind him are represented by the three on the right. The figure is surrounded by a group of trees, all in full bloom. The sun is in the sky above him. The figure
